# Оценка тональности

**Описание задания**

В рамках этого задания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является ли отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов  в поступившем нам на проверку отзыве больше.


Мы будем работать по заранее определённому пайплайну:

1.  Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных  и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше)  и 10 отзывов для проверки качества.   2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

2. Токенизируйте слова,  приведите их к нижнему регистру и к начальной форме  (1 балл за токенизацию, 1 - за начальную форму)

3. Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

4. Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1  - за коректно работающую функцию, 1 - за подсчёт accuracy)

5. Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек  - просто словами, писать улучшающий код не надо (1 балл)

6. Логичность и чистота кода (1 балл) 

Да, тут 11 баллов - один можно потерять.


В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачьтём полный балл

## 1. Парсинг* и токенизация

\*К сожалению, вайлдберис не закрасивосупился, поэтому копипастила 35 положительных и 35 отрицательных со следующих сайтов:
- [SYNERGETIC Гель для стирки универсальный](https://www.wildberries.ru/catalog/14792609/detail.aspx?targetUrl=XS)
- [SYNERGETIC Гель для стирки детского белья](https://www.wildberries.ru/catalog/14792610/detail.aspx?targetUrl=XS)
- [SYNERGETIC Кондиционер для белья "Миндальное молочко" ](https://www.wildberries.ru/catalog/14793781/detail.aspx?targetUrl=XS)
- [SYNERGETIC Гель для стирки 2в1](https://www.wildberries.ru/catalog/27530245/detail.aspx?targetUrl=RG)

Отзывы сохранены в файле posts.txt

In [69]:
import nltk
nltk.download("punkt")
# Не razel, поскольку не литературных текст с оч кривой пунктуацией

[nltk_data] Downloading package punkt to /Users/tasia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [112]:
posts = {"+": [], "-": []}

with open("posts.txt", encoding='utf-8') as f:
    for line in f:
        tag = line[0]
        post = [token for token in nltk.word_tokenize(line[2:-1].lower()) if token.isalpha()]
        posts[tag].append(post)

print("Число положительных отзывов:", len(posts["+"]))
print("Число отрицательных отзывов:", len(posts["-"]))

Число положительных отзывов: 35
Число отрицательных отзывов: 35


## 2. Создание множеств положительных и отрицательных слов

In [113]:
# Разделим на обучающую и тестовую выборки
pos_train = posts["+"][:29]
neg_train = posts["-"][:29]
all_test = {"+": posts["+"][30:], "-": posts["-"][30:]}

In [114]:
pos_all_words = {word for post in pos_train for word in post}
neg_all_words = {word for post in neg_train for word in post}
same_words = pos_all_words & neg_all_words

In [115]:
print("Всего слов в положительных отзывах:", len(pos_all_words))
print("Всего слов в отрицательных отзывах:", len(neg_all_words))
print("Общих слов:", len(same_words))

Всего слов в положительных отзывах: 230
Всего слов в отрицательных отзывах: 272
Общих слов: 58


In [116]:
pure_pos_words = pos_all_words - same_words
pure_neg_words = neg_all_words - same_words
print("Осталось положительных:", len(pure_pos_words))
print("Осталось отрицательных:", len(pure_neg_words))

Осталось положительных: 172
Осталось отрицательных: 214


## 3. Функция оценки тональности

In [117]:
def sentiment_analysis(post):
    pos_count = 0
    neg_count = 0
    
    for word in post:
        if word in pure_pos_words:
            pos_count += 1
        elif word in pure_neg_words:
            neg_count += 1
            
    if pos_count > neg_count:
        return "+"
    elif neg_count > pos_count:
        return "-"
    else:
        return "?"

## 4. Оценка качества

Поскольку формально у меня получилась не бинарная классификация, то и классическое accuracy использовать нельзя. Поэтому дальше под accuracy я считала отношение правильно_размеченных к хоть_как-то_размеченным.

In [121]:
t_pos_f_neg = 0 
t_neg_f_pos = 0
na = 0

for tag in ["+", "-"]:
    for post in all_test[tag]:
        ans = sentiment_analysis(post)
        if ans == "?":
            na += 1
        elif ans == tag:
            t_pos_f_neg += 1
        else:
            t_neg_f_pos +=1   

In [122]:
accuracy = t_pos_f_neg / (t_pos_f_neg + t_neg_f_pos)
covered = (t_pos_f_neg + t_neg_f_pos) / (t_pos_f_neg + t_neg_f_pos + na)
print("Accuracy:", accuracy)
print("Покрытие:", covered)

Accuracy: 1.0
Покрытие: 1.0


Удивительно, но наш разметчик работает великолепно... Но, возможно, что просто тестовая выборка очень маленькая. Да и отзывы довольно однотипные в целом.

## Идеи для улучшения

- Лемматизировать или стеммизировать, чтобы 100500 *отлично*, *отличный* не различать
- Не отделять отрицательные частицы от вершин
- Считать по биграммам (расширенный вариант предыдущего улучшения)
- Анализировать знаки препинания (скобочки, например)